[Reference](https://jnikenoueba.medium.com/real-time-applications-with-websockets-and-fastapi-7f9ea66bcddf)

```
fastapi_websockets/
├── app/
│   ├── __init__.py
│   ├── main.py
├── index.html
├── requirements.txt
└── README.md
```

# 1. Setting Up Your Environment
```
python -m venv venv
source venv/bin/activate  # On Windows use `venv\Scripts\activate`
pip install fastapi uvicorn
```

# 2. Creating a Simple WebSocket Server
```
main.py
```

In [2]:
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from typing import List

app = FastAPI()

class ConnectionManager:
    def __init__(self):
        self.active_connections: List[WebSocket] = []

    async def connect(self, websocket: WebSocket):
        await websocket.accept()
        self.active_connections.append(websocket)

    def disconnect(self, websocket: WebSocket):
        self.active_connections.remove(websocket)

    async def send_message(self, message: str, websocket: WebSocket):
        await websocket.send_text(message)

    async def broadcast(self, message: str):
        for connection in self.active_connections:
            await connection.send_text(message)

manager = ConnectionManager()

@app.websocket("/ws/{client_id}")
async def websocket_endpoint(websocket: WebSocket, client_id: int):
    await manager.connect(websocket)
    try:
        while True:
            data = await websocket.receive_text()
            await manager.send_message(f"Client {client_id}: {data}", websocket)
            await manager.broadcast(f"Client {client_id} says: {data}")
    except WebSocketDisconnect:
        manager.disconnect(websocket)
        await manager.broadcast(f"Client {client_id} disconnected")

# 3. Running the Application
```
uvicorn app.main:app --reload --host 0.0.0.0 --port 8000
```

# 4. Creating a Frontend for Real-Time Communication
```
index.html

<!DOCTYPE html>
<html>
<head>
    <title>WebSocket Chat</title>
</head>
<body>
    <h1>WebSocket Chat</h1>
    <input type="text" id="messageInput" autocomplete="off"/>
    <button onclick="sendMessage()">Send</button>
    <ul id="messages"></ul>

    <script>
        const clientId = Math.floor(Math.random() * 1000);
        const ws = new WebSocket(`ws://localhost:8000/ws/${clientId}`);

        ws.onmessage = function(event) {
            const messages = document.getElementById('messages');
            const message = document.createElement('li');
            message.textContent = event.data;
            messages.appendChild(message);
        };

        function sendMessage() {
            const input = document.getElementById("messageInput");
            ws.send(input.value);
            input.value = '';
        }
    </script>
</body>
</html>
```

# 5. Serving the Frontend with FastAPI

In [3]:
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from fastapi.responses import HTMLResponse
from typing import List

app = FastAPI()

class ConnectionManager:
    def __init__(self):
        self.active_connections: List[WebSocket] = []

    async def connect(self, websocket: WebSocket):
        await websocket.accept()
        self.active_connections.append(websocket)

    def disconnect(self, websocket: WebSocket):
        self.active_connections.remove(websocket)

    async def send_message(self, message: str, websocket: WebSocket):
        await websocket.send_text(message)

    async def broadcast(self, message: str):
        for connection in self.active_connections:
            await connection.send_text(message)

manager = ConnectionManager()

@app.websocket("/ws/{client_id}")
async def websocket_endpoint(websocket: WebSocket, client_id: int):
    await manager.connect(websocket)
    try:
        while True:
            data = await websocket.receive_text()
            await manager.send_message(f"Client {client_id}: {data}", websocket)
            await manager.broadcast(f"Client {client_id} says: {data}")
    except WebSocketDisconnect:
        manager.disconnect(websocket)
        await manager.broadcast(f"Client {client_id} disconnected")

@app.get("/")
async def get():
    with open("index.html") as f:
        return HTMLResponse(f.read())

# 6. Testing the Real-Time Application
```
uvicorn app.main:app --reload --host 0.0.0.0 --port 8000
```